In [1]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [2]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
# host = 'www.eye-ai.org'
host = 'dev.eye-ai.org'

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-06-18 10:45:47,176 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-18 10:45:47,177 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [4]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-BH9A" # rid dev:2-BH9A prod: 2-C61G


In [5]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-06-18 10:45:47,218 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-18 10:45:47,219 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [6]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

2024-06-18 10:45:48,834 - INFO - File [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_df_join.json] transfer successful. 0.68 KB transferred. Elapsed time: 0:00:00.000108.
2024-06-18 10:45:48,835 - INFO - Verifying SHA256 checksum for downloaded file [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_df_join.json]
2024-06-18 10:45:48,850 - INFO - Configuration validation successful!


DataPathException: DETAIL:  Key (Dataset)=(2-7P5P) is not present in table "Dataset".

409 Client Error: CONFLICT for url: [https://dev.eye-ai.org/ermrest/catalog/eye-ai/entity/eye-ai:Dataset_Execution?defaults=RMB,RID,RCB,RMT,RCT] Details: b'Request conflicts with state of server. Detail: Input data violates model. ERROR:  insert or update on table "Dataset_Execution" violates foreign key constraint "Dataset_Execution_Dataset_fkey"\nDETAIL:  Key (Dataset)=(2-7P5P) is not present in table "Dataset".\n\n'

In [8]:
# Extract and join four modalities from the multimodal dataset
modality_df = EA.extract_modality(configuration_records.bag_paths[0])
modality_df # returns the paths to four csv file for each modality (Clinical data, HVF reports, RNFL reports, fundus image)

{'Clinic': PurePosixPath('/data/lizhiwei/EyeAI_working/clinic.csv'),
 'HVF': PurePosixPath('/data/lizhiwei/EyeAI_working/HVF.csv'),
 'RNFL': PurePosixPath('/data/lizhiwei/EyeAI_working/RNFL.csv'),
 'Image': PurePosixPath('/data/lizhiwei/EyeAI_working/image.csv')}

In [11]:
data_path = configuration_records.bag_paths[0]

subject = pd.read_csv(data_path/'data/Subject.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
observation = pd.read_csv(data_path/'data/Observation.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
image = pd.read_csv(data_path/'data/Image.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
clinic = pd.read_csv(data_path/'data/Clinical_Records.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
observation_clinic_asso = pd.read_csv(data_path/'data/Observation_Clinic_Asso.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
icd10 = pd.read_csv(data_path/'data/Clinic_ICD10.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
icd10_asso = pd.read_csv(data_path/'data/Clinic_ICD_Asso.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
report = pd.read_csv(data_path/'data/Report.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
RNFL_OCR = pd.read_csv(data_path/'data/RNFL_OCR.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
HVF_OCR = pd.read_csv(data_path/'data/HVF_OCR.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])






In [12]:
subject


,RID,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,date_of_encounter,provider,reviewed_date,dr_level,...,Subject_Ethnicity,Subject_hypertension,Subject_insulin_dependent,Subject_pregnant,Subject_cataract,Subject_maculopathy,Subject_other,Subject_image_quality,site_mrn,Consultant
0,2-7KTR,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2-7KTT,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2-7KTW,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69WJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2-7KTY,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69WM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2-7KV0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69WJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,2-7P5C,1195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1195,2-7P5E,1196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69WM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,2-7P5G,1197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,2-7P5J,1198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69WE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
pd.read_csv(modality_df['Clinic'])

,RID_Subject,Subject_ID,Gender,Ethnicity,RID_Observation,Observation_ID,Date_of_Encounter_Observation,RID_Clinic,Date_of_Encounter_Clinic,LogMAR_VA,...,IOP,Refractive_Error,CCT,CDR,Gonioscopy,Condition_Display,Provider,Clinical_ID,Side,Label
0,2-7KTR,1,F,Caucasian,2-7RGP,1,2020-08-05 00:00:00+00,2-7XT4,2020-08-05 00:00:00,0.14,...,13.0,NaN,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,3130.0,Left,POAG
1,2-7KTR,1,F,Caucasian,2-7RGP,1,2020-08-05 00:00:00+00,2-7XT6,2020-08-05 00:00:00,0.22,...,14.0,NaN,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,3131.0,Right,POAG
2,2-7KTR,1,F,Caucasian,2-7RGR,2,2020-10-22 00:00:00+00,2-7XT8,2020-10-22 00:00:00,0.00,...,14.0,NaN,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,3132.0,Left,POAG
3,2-7KTR,1,F,Caucasian,2-7RGR,2,2020-10-22 00:00:00+00,2-7XTA,2020-10-22 00:00:00,0.00,...,13.0,NaN,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,3133.0,Right,POAG
4,2-7KTT,2,M,Caucasian,2-7RGT,659,2023-04-18 00:00:00+00,2-7XTC,2023-04-18 00:00:00,0.18,...,9.0,-0.5,NaN,0.70,4/4/4/4,"Primary open-angle glaucoma, bilateral, modera...",Van Nguyen,52847.0,Left,POAG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4970,2-7P5M,1199,F,Asian,2-7XFM,474,2023-04-19 00:00:00+00,2-868W,2023-04-19 00:00:00,0.18,...,19.0,NaN,NaN,0.85,NaN,Wilson's disease_ Type 2 diabetes mellitus wit...,Brian Song,60014.0,Right,GS
4971,2-7P5M,1199,F,Asian,2-7XFP,473,2022-10-19 00:00:00+00,2-868Y,2022-10-19 00:00:00,0.18,...,16.0,NaN,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,44728.0,Left,GS
4972,2-7P5M,1199,F,Asian,2-7XFP,473,2022-10-19 00:00:00+00,2-8690,2022-10-19 00:00:00,0.22,...,16.0,NaN,NaN,0.85,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,44729.0,Right,GS
4973,2-7P5M,1199,F,Asian,2-7XFR,472,2022-04-20 00:00:00+00,2-8692,2022-04-20 00:00:00,0.18,...,14.0,NaN,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,44726.0,Left,GS


In [ ]:
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)
